In [2]:
'''
These are the important imports and functions used in the detection part of the project
assuming the other libraries are already installed and imported like cv2, numpy, etc.
'''
from fastai.vision.all import *
from fastai.metrics import accuracy
from fastseg import MobileV3Small
import torch

import cv2
import numpy as np
from scipy.spatial import distance

In [3]:
# The functions needed for the detection part
def get_pred_for_mobilenet(model, img_array): # this is to make sure that the model is in the GPU
    with torch.no_grad():
        # image_tensor = np.expand_dims(img_array, -1).transpose(2, 0, 1).astype('float32') / 255
        image_tensor = img_array.transpose(2, 0, 1).astype('float32') / 255
        x_tensor = torch.from_numpy(image_tensor).to("cuda").unsqueeze(0)
        model_output = F.softmax(model.forward(x_tensor), dim=1).cpu().numpy()
    return model_output

def extract_lane_boundaries(left_mask, right_mask, threshold=0.3):
    left_boundary = (left_mask > threshold).astype(np.uint8)
    right_boundary = (right_mask > threshold).astype(np.uint8)

    contours, _ = cv2.findContours(left_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        left_boundary = cv2.drawContours(left_boundary, contours, -1, (1), thickness=cv2.FILLED)

    contours, _ = cv2.findContours(right_boundary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        right_boundary = cv2.drawContours(right_boundary, contours, -1, (1), thickness=cv2.FILLED)

    return left_boundary, right_boundary

def lane_detection_pipeline(img, model_path="seg_model.pth"):
    # Load the model
    model = torch.load(model_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Get predictions
    back, left, right = get_pred_for_mobilenet(model, img)[0]

    # Extract left and right boundaries
    left_boundary, right_boundary = extract_lane_boundaries(left, right)

    # Create ROI mask
    # roi_mask = (left_boundary | right_boundary).astype(np.uint8)

    # # Create a copy of the original image
    # image_with_roi = np.copy(img)

    # # Set all pixels outside the ROI to black
    # image_with_roi[roi_mask == 0] = [0, 0, 0]

    # # Overlay the left and right lane boundaries in red color
    # image_with_roi[left_boundary > 0] = [255, 0, 0]
    # image_with_roi[right_boundary > 0] = [0, 255, 0]

    # Find the coordinates of the left and right boundaries
    left_y, left_x = np.where(left_boundary > 0)
    right_y, right_x = np.where(right_boundary > 0)

    # Create a black background mask with the same dimensions as the image
    mask = np.zeros_like(img, dtype=np.uint8)

    # Create a polygon for the left boundary
    left_polygon = np.column_stack((left_x, left_y))

    # Create a polygon for the right boundary (reverse the order to close the gap)
    right_polygon = np.column_stack((right_x[::-1], right_y[::-1]))

    # Combine the left and right polygons to fill the region between them
    polygon_points = np.vstack((left_polygon, right_polygon))

    # Use fillPoly to fill the region between the left and right boundaries
    try:
        cv2.fillPoly(mask, [polygon_points], (255, 255, 255))
    except Exception as e:
        print(f"Error in cv2.fillPoly: {e}")
        return None

    # Bitwise-AND your original image and the mask
    result = cv2.bitwise_and(img, mask)

    # Save the resulting image
    # cv2.imwrite(save_path, result)
    return result, left, right

    # return result


In [8]:
def warp_img(img):
    # https://nikolasent.github.io/opencv/2017/05/07/Bird's-Eye-View-Transformation.html
    IMAGE_H = img.shape[0]
    IMAGE_W = img.shape[1]
    src = np.float32(
        [[0, IMAGE_H], [IMAGE_W, IMAGE_H], [0, IMAGE_H // 10], [IMAGE_W, IMAGE_H // 10]]
    )
    dst = np.float32(
        [[IMAGE_W // 2.8, IMAGE_H], [IMAGE_W // 1.8, IMAGE_H], [0, 0], [IMAGE_W, 0]]
    )
    img = img[int(IMAGE_H // 2):IMAGE_H, :]  # Apply np slicing for ROI crop
    M = cv2.getPerspectiveTransform(src, dst)  # The transformation matrix
    img = cv2.warpPerspective(img, M, (IMAGE_W, IMAGE_H))  # Image warping
    img = img[
        int(IMAGE_H // 10) : int(IMAGE_H // 1.3),
        int(IMAGE_W // 3) : int(IMAGE_W // 1.7),
    ]
    return img

def warp_img2(img):
    IMAGE_H = img.shape[0]
    IMAGE_W = img.shape[1]
    new_h = 640
    new_w = 300
    src = np.float32(
        [
            [0, IMAGE_H], 
            [IMAGE_W , IMAGE_H], 
            [int(IMAGE_W // 2.2), int(IMAGE_H // 1.7)], 
            [int(IMAGE_W // 1.8), int(IMAGE_H // 1.7)],
        ]
    )
    dst = np.float32(
        [
            [0, new_h], 
            [new_w, new_h], 
            [0, 0], 
            [new_w, 0],
        ]
    )

    M = cv2.getPerspectiveTransform(src, dst)  # The transformation matrix
    img = cv2.warpPerspective(img, M, (new_w, new_h))  # Image warping
    return img

def gaussian_blur(img, kernel_size=(3, 3)):
    return cv2.GaussianBlur(img, kernel_size, 0)

def canny_edge(img, low_threshold=100, high_threshold=200):
    return cv2.Canny(img, low_threshold, high_threshold)

def harris_corner(img):
    # img = np.float32(img)
    corners = cv2.cornerHarris(img, blockSize=2, ksize=3, k=0.04)
    # corners = cv2.dilate(corners, None)
    # img[corners > 0.01 * corners.max()] = 255
    return corners

def dilation(img, kernel_size=(3, 3)):
    return cv2.dilate(img, np.ones(kernel_size, np.uint8))

def clahe(img, kernel_size=(3, 3)):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(img)
    return clahe_image

def detect_hough_lines(img):
    return cv2.HoughLinesP(
        img, rho=1, theta=np.pi / 180, threshold=20, minLineLength=5, maxLineGap=10
    )


def draw_hough_lines(img, lines):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), 255, 3)
    return img


def detect_correct_mark(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.minAreaRect(contour) for contour in contours]

    # correct_rects = [rect for rect in rects if min(rect[1][1], rect[1][0]) < img.shape[1] // 2]
    # print(f"# of correct rects: {len(correct_rects)}")

    # d1 = rect[1][0]
    # d2 = rect[1][1]
    # width = min(rect[1][1], rect[1][0])
    # height = max(rect[1][1], rect[1][0])

    center_bottom = (img.shape[1] // 2, img.shape[0] // 1.2)
    distances = [distance.euclidean(rect[0], center_bottom) for rect in rects]
    btm_cntr_rect_idx = np.argmin(distances)

    return rects, btm_cntr_rect_idx


def draw_rectangle_features(img, rect, idx):
    width = min(rect[1][1], rect[1][0])
    box = np.intp(cv2.boxPoints(rect))

    top_left = box[np.argmax(box[:, 1])] - np.array([0, 50])
    bottom_left = box[np.argmin(box[:, 1])] - np.array([0, 50])

    cv2.drawContours(img, [box], 0, (255, 0, 0), 2)

    cv2.circle(
        img,
        center=tuple(map(int, rect[0])),
        radius=3,
        color=(255, 255, 255),
        thickness=5,
    )

    cv2.putText(
        img,
        text=str(idx),
        org=tuple(map(int, top_left)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=2,
        color=(255, 255, 255),
        thickness=3,
    )
    return img


def map_values(
    rect: tuple, img: np.ndarray, car_steer: float = 0
) -> tuple[float, float]:
    """
    Map the values for the steer to (-1, 1)
    and the values for the throttle to (0, 1)

    Parameters
    ----------
    rect: tuple
        the rectangle that is used to determine the throttle and steering angle
    img: np.ndarray
        the image that is used to determine the mapping
    Return
    ------
    throttle: float
        the throttle for the car
    steer: float
        the steering angle for the car
    """
    center = rect[0]
    img_center = (img.shape[1] // 2, img.shape[0] // 1.2)

    offset = center[0] - img_center[0]  # offset from the center of the image
    offset = offset / img.shape[1]  # normalize the offset

    d1 = rect[1][0]
    d2 = rect[1][1]
    width = min(rect[1][1], rect[1][0])
    height = max(rect[1][1], rect[1][0])
    angle = rect[2]

    # rounding to the nearest 5
    width = int(5 * round(width / 5))
    angle = int(5 * round(angle / 5))

    if angle in (0, 90, -0, -90, 0.0, 90.0, -0.0, -90.0):
        angle = 0

    elif d1 < d2:
        angle = 90 - angle

    else:
        angle = -angle

    throttle = max(width / (120 + car_steer), 0.4)  # A trial and error value
    steer = angle / (90 + throttle * 100) + (offset)

    return throttle, steer

In [9]:
def show_imgs(imgs, titles):
    rows = int(np.ceil(len(imgs) / 3 ))
    cols = 3
    figsize = (cols * 5, rows * 4)
    _, axs = plt.subplots(rows, cols, figsize=figsize)
    for img, title, ax in zip(imgs, titles, axs.flatten()):
        ax.imshow(img, cmap="gray")
        ax.set_title(title)
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [12]:
def show_process_image(img):
    img = warp_img(img)
    img_warp = img.copy()
    img = gaussian_blur(img)
    # img = harris_corner(img)
    img = canny_edge(img)
    img_canny = img.copy()
    img = dilation(img)
    img_w, img_h = img.shape[1], img.shape[0]

    lines = detect_hough_lines(img)
    if lines is None:
        return img, np.zeros((img_h, img_w), dtype=np.uint8)

    img_hou = np.zeros((img_h, img_w), dtype=np.uint8)
    img_hou = draw_hough_lines(img_hou, lines)

    rects, bottom_center_rect_idx = detect_correct_mark(img_hou)

    throttle, steer = map_values(rects[bottom_center_rect_idx], img_hou)
    # return throttle, steer

    img_hou = cv2.cvtColor(img_hou, cv2.COLOR_GRAY2RGB)

    center_bottom = (img_hou.shape[1] // 2, img_hou.shape[0] // 1.5)
    cv2.circle(
        img_hou,
        center=tuple(map(int, center_bottom)),
        radius=50,
        color=(0, 0, 255),
        thickness=10,
    )
    for i, rect in enumerate(rects):
        img_hou = draw_rectangle_features(img_hou, rect, idx=i)
    
    cv2.putText(
        img_hou,
        text=f"{bottom_center_rect_idx} SELECTED",
        org=(0, 50),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=1,
        color=(0, 255, 255),
        thickness=3,
    )
    
    return img_warp, img_hou

In [16]:
cap = cv2.VideoCapture("../imgs/project_video.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord("q") or ret == False:
        break
    result, left, right = lane_detection_pipeline(frame)
    # result_warped, result_hou = show_process_image(result)
    cv2.imshow("Frame", frame)
    cv2.imshow("Lane", result)
    # cv2.imshow("Warped", result_hou)
    # time.sleep(0.1)

cap.release()
cv2.destroyAllWindows()